In [ ]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage , HumanMessage
from langchain_openai import ChatOpenAI 
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

from langgraph.prebuilt import ToolNode , tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool

import requests
import random


In [ ]:
load_dotenv()

In [ ]:
model = ChatOpenAI()

In [ ]:
#Tools
search_tool = DuckDuckGoSearchRun(region="us-en")

@tool
def calculator(first_num: float, second_num: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.
    Supported operations: add, sub, mul, div
    """
    try:
        if operation == "add":
            result = first_num + second_num
        elif operation == "sub":
            result = first_num - second_num
        elif operation == "mul":
            result = first_num * second_num
        elif operation == "div":
            if second_num == 0:
                return {"error": "Division by zero is not allowed"}
            result = first_num / second_num
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        
        return {"first_num": first_num, "second_num": second_num, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}
    
@tool
def get_stock_price(symbol: str) -> dict:
    """
    Fetch latest stock price for a given symbol (e.g. 'AAPL', 'TSLA') 
    using Alpha Vantage with API key in the URL.
    """
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=C9PE94QUSD9T8WS08"
    r = requests.get(url)
    return r.json


In [ ]:
#Make tools list
tools = [search_tool , calculator , get_stock_price]

#Now make Model call
model_with_tools = model.bind_tools(tools)

In [ ]:
#Define State
class ChatState(TypedDict):
  messages : Annotated[list[BaseMessage],add_messages]

In [ ]:
#Graph node function
def chat_node(state : ChatState):
  """model node that may answer on request a tool call"""
  messages = state['messages']
  response = model_with_tools.invoke(messages)

  return {"messages": [response]}

tool_node = ToolNode(tools)

In [ ]:
#Graph Structure
graph  =  StateGraph(ChatState)

graph.add_node('chat_node', chat_node)
graph.add_node('tools', tool_node)

graph.add_edge(START , "chat_node")

# If the LLM asked for a tool, go to ToolNode; else finish
graph.add_conditional_edges("chat_node" , tools_condition)

graph.add_edge("tools" , "chat_node")


In [ ]:
chatbot = graph.compile()

chatbot

In [ ]:
#regular chat
final_output = chatbot.invoke({"messages": [HumanMessage(content="Hello!")]})

print(final_output["messages"][-1].content)

In [ ]:

# Chat requiring tool
output = chatbot.invoke({"messages": [HumanMessage(content="What is 56*46?")]})
print(output["messages"][-1].content)

In [ ]:

# Chat requiring tool
result = chatbot.invoke({"messages": [HumanMessage(content="What is the stock price of apple")]})
print(result["messages"][-1].content)

In [ ]:
# Chat requiring tool
out = chatbot.invoke({"messages": [HumanMessage(content="First find out the stock price of Apple using get stock price tool then use the calculator tool to find out how much will it take to purchase 50 shares?")]})
print(out["messages"][-1].content)